In [1]:
import configparser

from sqlalchemy import create_engine

import pandas as pd

import os

import pandas_gbq

from datetime import datetime

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'dummy-surveillance-project-6915f229d012'

In [3]:
parser = configparser.ConfigParser()

parser.read('configuration file.cfg')

hostname = parser.get('mysql_config', 'hostname')

port = parser.get('mysql_config', 'port')

username = parser.get('mysql_config', 'username')

dbname = parser.get('mysql_config', 'database')

password = parser.get('mysql_config', 'password')

In [4]:
query_string = "SELECT * FROM `dummy-surveillance-project.demo.orderTimestamp` "

timestamp_bq_df = pandas_gbq.read_gbq(query_string, project_id = 'dummy-surveillance-project')

Downloading: 100%|██████████|


In [5]:
try:
    
    latest_timestamp = timestamp_bq_df['orderTimestamp'][0].strftime('%Y-%m-%d %H:%M:%S')
    
except IndexError:
    
    latest_timestamp = '1900-01-01 00:00:00'

In [6]:
sqlEngine = create_engine(f'mysql://{username}:{password}@{hostname}:{port}/{dbname}')

orders_mysql_df = pd.read_sql(f"SELECT * FROM orders WHERE orderTimestamp > '{latest_timestamp}';", con = sqlEngine)

In [7]:
query_string2 = "SELECT * FROM `dummy-surveillance-project.demo.orders` "

orders_bq_df = pandas_gbq.read_gbq(query_string2, project_id = 'dummy-surveillance-project')

Downloading: 100%|██████████|


In [8]:
bq_table_schema = [
    
    {'name':'orderId', 'type':'INTEGER'},
    
    {'name':'orderValue', 'type':'FLOAT'},
    
    {'name':'Timestamp', 'type':'TIMESTAMP'}
    
]

bq_table_schema2 = [{'name':'Timestamp', 'type':'TIMESTAMP'}]

In [9]:
if orders_bq_df.empty:
    
    orders_mysql_df.to_gbq('demo.orders', if_exists = 'append', table_schema = bq_table_schema,
                        
                        project_id = 'dummy-surveillance-project')
    
    
else:
    
    for i in orders_mysql_df['orderId']:
        
        if i in orders_bq_df['orderId']:
            
            updated_orderValue = orders_mysql_df[orders_mysql_df['orderId']==i]['orderValue'].iloc[0]
            
            updated_orderTimestamp = orders_mysql_df[orders_mysql_df['orderId']==i]['orderTimestamp'].iloc[0]
            
            query_string3 = f"UPDATE `dummy-surveillance-project.demo.orders` SET orderValue = {updated_orderValue}, \
            orderTimestamp = '{updated_orderTimestamp}' WHERE orderId = {i}"  
            
            orders_bq_df = pandas_gbq.read_gbq(query_string3, project_id = 'dummy-surveillance-project')
            
        else:
            
            incremental_row_df = orders_mysql_df[orders_mysql_df['orderId']==i]
            
            incremental_row_df.to_gbq('demo.orders', if_exists = 'append', table_schema = bq_table_schema, 
                                      
                                      project_id = 'dummy-surveillance-project')        

100%|██████████| 1/1 [00:00<?, ?it/s]


In [10]:
order_Timestamp_max_mysql_df = pd.read_sql('SELECT MAX(orderTimestamp) orderTimestamp FROM orders', con = sqlEngine)

sqlEngine.dispose()

In [11]:
order_Timestamp_max_mysql_df.to_gbq('demo.orderTimestamp', if_exists = 'replace', table_schema = bq_table_schema2, 
           
           project_id = 'dummy-surveillance-project')

100%|██████████| 1/1 [00:00<?, ?it/s]
